## Function Calling

In [1]:
from openai import AzureOpenAI
client = AzureOpenAI()

In [2]:
response = client.chat.completions.create(model='myllm',
                                          messages=[{"role":"user","content":"hi"}])
print(response.choices[0].to_json(indent=2))

{
  "finish_reason": "stop",
  "index": 0,
  "logprobs": null,
  "message": {
    "content": "Hello! How can I assist you today?",
    "refusal": null,
    "role": "assistant",
    "annotations": []
  },
  "content_filter_results": {
    "hate": {
      "filtered": false,
      "severity": "safe"
    },
    "self_harm": {
      "filtered": false,
      "severity": "safe"
    },
    "sexual": {
      "filtered": false,
      "severity": "safe"
    },
    "violence": {
      "filtered": false,
      "severity": "safe"
    }
  }
}


In [3]:
import os
import json
import requests

In [10]:
def get_current_weather(city:str)->dict:
    """ this function can be used to fetch weather information for a given city"""
    api_key=os.environ['OPENWEATHERMAP_API_KEY']

    url = f"https://api.openweathermap.org/data/2.5/weather?q={city}&appid={api_key}"
    response = requests.get(url)
    response = response.content.decode() # bytes to string
    response = json.loads(response) # string to dict
    output = {"city":city,
              "weather":response['weather'][0]['description'],
              "temperature":response['main']['temp'],
              "unit":"kelvin"}
    return output


In [11]:
get_current_weather("mumbai")

{'city': 'mumbai', 'weather': 'haze', 'temperature': 302.14, 'unit': 'kelvin'}

#### There 3 important components to be provided to LLM as metadata about functions
1. function name: should be descriptive, it should define what function does, this does not have to be same as the actual name of function
2. function description: should be descriptive, if possible include example of calling, - core basis on which LLM relies on selecting the right function
3. Arguments: list of params the function supports with example,description of each parameter

In [12]:
tools = [{"type":"function",
          "function":{
              "name":"get_current_weather",
              "description":"this function can be used to current weather information for a given city.",
              "parameters":{
                  "type":"object",
                  "properties":{"city":{"type":"string","description":"name of city/location. e.g. mumbai, new york"},
                                },
                                "required":['city'],
              }
          }}]

In [21]:
tool_names = {"get_current_weather":get_current_weather} # name of function in metadata : name of object

def generate_response(prompt):
    messages = [{"role":"system","content":"you are a helpful assistant, who answers questions in concise way"},
                {"role":"user","content":prompt},]
    
    first_response = client.chat.completions.create(model="myllm",messages=messages,
                                                    temperature=0.1,tools=tools,
                                                    tool_choice='auto')
    
    first_response = first_response.choices[0].message
    if first_response.tool_calls:
        # do something
        print(first_response)
        messages.append(first_response)

        # iterate through all the functions, execute and log responses in messages
        for tool in first_response.tool_calls:
            tool_name = tool.function.name
            tool_args = json.loads(tool.function.arguments) # json.loads converts str to dict
            fun_toexecute = tool_names[tool_name]
            tool_output = fun_toexecute(**tool_args)
            messages.append({"tool_call_id":tool.id,
                             "role":"tool","name":tool_name,"content":json.dumps(tool_output)})
        second_response = client.chat.completions.create(model="myllm",messages=messages,temperature=0.1)
        return second_response.choices[0].message.content
        
    else:
        return first_response.content

In [22]:
op = generate_response("what is quantum computing")
print(op)

Quantum computing is a type of computation that utilizes the principles of quantum mechanics to process information. Unlike classical computers, which use bits as the smallest unit of data (0s and 1s), quantum computers use quantum bits, or qubits. Qubits can exist in multiple states simultaneously due to superposition, allowing quantum computers to perform complex calculations at much higher speeds than classical computers for certain tasks.

Key concepts in quantum computing include:

1. **Superposition**: Qubits can represent both 0 and 1 at the same time, enabling parallel processing.
2. **Entanglement**: Qubits can be interconnected in such a way that the state of one qubit can depend on the state of another, no matter the distance between them.
3. **Quantum gates**: Operations that manipulate qubits, similar to logic gates in classical computing.

Quantum computing has the potential to revolutionize fields such as cryptography, optimization, and drug discovery by solving problems

In [23]:
op = generate_response("what is the current weather in new york today.")
print(op)

ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_rbCUqjD3X8IJDFU0FsYfyYXZ', function=Function(arguments='{"city":"new york"}', name='get_current_weather'), type='function')])
The current weather in New York today is hazy, with a temperature of approximately 29.1°C (84.4°F).


In [24]:
op = generate_response("what is the current weather in ahmedabad today.")
print(op)###

ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_030lYemn09PQybdPz05l0LfX', function=Function(arguments='{"city":"ahmedabad"}', name='get_current_weather'), type='function')])
The current weather in Ahmedabad is smoky, with a temperature of approximately 33°C (306.17 K).


In [25]:
op = generate_response("what is the current weather in ahmedabad  and Jaiput today.")
print(op)

ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_eL5vIuYMbRWzmYtPjzcCYfgz', function=Function(arguments='{"city": "Ahmedabad"}', name='get_current_weather'), type='function'), ChatCompletionMessageToolCall(id='call_EhgNfQ023deH6m1fAq9RmmLw', function=Function(arguments='{"city": "Jaipur"}', name='get_current_weather'), type='function')])
The current weather is as follows:

- **Ahmedabad**: Smoke, Temperature: 304.17 K (approximately 31°C)
- **Jaipur**: Haze, Temperature: 307.77 K (approximately 35.6°C)


## Function calling: ADVANCE

In [26]:
import wikipedia

In [27]:
def wiki(query:str)->str:
    response = wikipedia.summary(query)
    return response

In [ ]:
tool_names = {"get_current_weather":get_current_weather,
              "get_wikipedia_summary":wiki} # name of function in metadata : name of object


In [29]:
tools = [{"type":"function",
          "function":{
              "name":"get_current_weather",
              "description":"this function can be used to current weather information for a given city.",
              "parameters":{
                  "type":"object",
                  "properties":{"city":{"type":"string","description":"name of city/location. e.g. mumbai, new york"},
                                },
                                "required":['city'],
              }
          }},
          {"type":"function",
          "function":{
              "name":"get_wikipedia_summary",
              "description":"this function can be used to get information about locations, places people and historical information for a given query.",
              "parameters":{
                  "type":"object",
                  "properties":{"query":{"type":"string","description":"name of city/location or event. e.g. illiois, world war II"},
                                },
                                "required":['query'],
              }
          }}]

In [36]:

def generate_response(prompt):
    messages = [{"role":"system","content":"you are a helpful assistant, who answers questions in concise way"},
                {"role":"user","content":prompt},]
    
    while True:
        first_response = client.chat.completions.create(model="myllm",messages=messages,
                                                        temperature=0.1,tools=tools,
                                                        tool_choice='auto')
        
        first_response = first_response.choices[0].message
        if first_response.tool_calls:
            # do something
            messages.append(first_response)

            # iterate through all the functions, execute and log responses in messages
            for tool in first_response.tool_calls:
                print(tool)
                tool_name = tool.function.name
                tool_args = json.loads(tool.function.arguments) # json.loads converts str to dict
                fun_toexecute = tool_names[tool_name]
                tool_output = fun_toexecute(**tool_args)
                messages.append({"tool_call_id":tool.id,
                                "role":"tool","name":tool_name,"content":json.dumps(tool_output)})            
        else:
            break
    return first_response.content

In [37]:
op = generate_response("what is the current weather in ahmedabad  and Jaipur today.")
print(op)

ChatCompletionMessageToolCall(id='call_wHgU9vDuCNJor4KslPaA4bwF', function=Function(arguments='{"city": "ahmedabad"}', name='get_current_weather'), type='function')
ChatCompletionMessageToolCall(id='call_NQtXaV4wu3tVt1SoXcXL8AbM', function=Function(arguments='{"city": "jaipur"}', name='get_current_weather'), type='function')
The current weather is as follows:

- **Ahmedabad**: Smoke, Temperature: 304.17 K
- **Jaipur**: Haze, Temperature: 306.77 K


In [39]:
op = generate_response("Tell me about 1st president of USA")
print(op)

ChatCompletionMessageToolCall(id='call_GMn1sZWpIBc0Pw819dlmuLPC', function=Function(arguments='{"query":"George Washington"}', name='get_wikipedia_summary'), type='function')
George Washington (February 22, 1732 – December 14, 1799) was the first president of the United States, serving from 1789 to 1797. He is known as the Father of the Nation for his leadership during the American Revolutionary War, where he commanded the Continental Army against British forces.

Born in Virginia, Washington became a military leader during the French and Indian War and later opposed British oppression of the American colonies. He was appointed commander-in-chief at the start of the Revolutionary War and achieved significant victories, including the Siege of Boston and the battles of Trenton and Princeton.

After the war, Washington presided over the Constitutional Convention in 1787, which drafted the U.S. Constitution. He was unanimously elected president in 1788 and 1792, establishing important prec

In [40]:
op = generate_response("What is the capital city of Tripura and whats the weather there.")
print(op)

ChatCompletionMessageToolCall(id='call_hgTdRRmxY553w3hBJ9YOKaVC', function=Function(arguments='{"query": "Tripura"}', name='get_wikipedia_summary'), type='function')
ChatCompletionMessageToolCall(id='call_LE5BF658752HvfbYmOsvfail', function=Function(arguments='{"city": "Agartala"}', name='get_current_weather'), type='function')
The capital city of Tripura is Agartala. Currently, the weather in Agartala is clear sky with a temperature of approximately 25.76°C (298.91 K).
